In [1]:
import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)
import os 
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler

from utils.afrr_preprocessing import preprocess_afrr_data
from ensamble_forecast.lr_forecaster import run_lr_pipeline
from ensamble_forecast.gp_forecaster import run_gp_pipeline
from ensamble_forecast.xgb_forecaster import run_xgb_pipeline

from utils.forecast_utils import get_forecast_params
from darts.timeseries import concatenate

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
data_path = "../data/afrr_price.parquet"
train_start = "2024-10-01 22:00:00"
test_start = "2025-01-09 22:00:00"
test_end = "2025-03-20 22:00:00"

In [ ]:
forecast_params = get_forecast_params()
output_chunk_length = forecast_params['output_chunk_length']
forecast_horizon = forecast_params['forecast_horizon']
stride = forecast_params['stride']
quantiles = forecast_params['quantiles']

# Run the pipeline with all parameters defined at the end
lr_models, lr_results = run_lr_pipeline(
    data_path=data_path,
    hyper_params_path= "../data/results/lr_hp_results.json",
    train_start=train_start,
    test_start=test_start,
    test_end=test_end,
    output_chunk_length=output_chunk_length,
    forecast_horizon=forecast_horizon,
    stride=stride,
    quantiles=quantiles
)
lr_results.to_parquet("lr_model_forecast.par")


### Gaussian process Models 

In [ ]:
forecast_params = get_forecast_params()
output_chunk_length = forecast_params['output_chunk_length']
forecast_horizon = forecast_params['forecast_horizon']
stride = forecast_params['stride']
quantiles = forecast_params['quantiles']

# Run the pipeline with all parameters defined at the end
gp_model, gp_results = run_gp_pipeline(
    data_path=data_path,
    hyper_params_path="../data/results/gp_hp_results.json",
    train_start=train_start,
    test_start=test_start,
    test_end=test_end,
    output_chunk_length=output_chunk_length,
    forecast_horizon=forecast_horizon,
    stride=stride,
    quantiles=quantiles
)

gp_results.to_parquet("gp_model_forecast.par")

### XgBoost Model

In [3]:
# Get forecast parameters
forecast_params = get_forecast_params()
output_chunk_length = forecast_params['output_chunk_length']
forecast_horizon = forecast_params['forecast_horizon']
stride = forecast_params['stride']

# Run the pipeline with all parameters defined at the end
xgb_model, xgb_results = run_xgb_pipeline(
    data_path=data_path,
    hyper_params_path="../data/results/xgb_hp_results.json",
    train_start=train_start,
    test_start=test_start,
    test_end=test_end,
    output_chunk_length=output_chunk_length,
    forecast_horizon=forecast_horizon,
    stride=stride)

#xgb_results.to_parquet("xgboost_model_forecast.par")

----------------------------------------
EMBEDDING SUMMARY:
Embedding dimension	:  120
Trajectory dimensions	: (120, 4390)
Complete dimension	: (120, 4390)
Missing dimension     	: (120, 0)
----------------------------------------
DECOMPOSITION SUMMARY:
Rank of trajectory		: 120
Dimension of projection space	: 120
Characteristic of projection	: 1.0


XGBoostError: [14:23:38] /Users/runner/work/xgboost/xgboost/src/objective/quantile_obj.cu:39: Check failed: info.labels.Shape(1) == 1 (24 vs. 1) : Multi-target is not yet supported by the quantile loss.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000000306638428 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x00000003068b1594 xgboost::obj::QuantileRegression::Targets(xgboost::MetaInfo const&) const + 460
  [bt] (2) 3   libxgboost.dylib                    0x00000003068334b0 xgboost::LearnerConfiguration::ConfigureTargets() + 368
  [bt] (3) 4   libxgboost.dylib                    0x00000003068312ec xgboost::LearnerConfiguration::ConfigureModelParamWithoutBaseScore() + 32
  [bt] (4) 5   libxgboost.dylib                    0x0000000306823570 xgboost::LearnerConfiguration::Configure() + 1308
  [bt] (5) 6   libxgboost.dylib                    0x000000030682379c xgboost::LearnerImpl::UpdateOneIter(int, std::__1::share

In [ ]:
if __name__ == "__main__":
    print(f"Environment check:")
    print(f" Darts {darts.__version__}")
    
    # Rest of your main code...

Environment check:


NameError: name 'xgboost' is not defined

In [8]:
# Add this at the start of your code to verify versions
import xgboost, darts
print(f"XGBoost version: {xgboost.__version__}")
print(f"Darts version: {darts.__version__}")

XGBoost version: 3.0.0
Darts version: 0.34.0


### Original Model

In [ ]:
(afrr_pr_ts_scl_train, 
afrr_pr_ts_scl_test, 
afrr_pr_ts_orig_train, 
afrr_pr_ts_orig_test, 
exog_ts_scl_train, 
exog_ts_scl_test,
afrr_pr_scaler
) = preprocess_afrr_data(data_path, train_start, test_start, test_end)

In [ ]:
afrr_pr_ts_orig = concatenate([afrr_pr_ts_orig_train, afrr_pr_ts_orig_test], axis=0)
afrr_pr_ts_orig = afrr_pr_ts_orig.with_columns_renamed(['aFRR_UpCapPriceEUR'], col_names_new=["afrr_up_cap_price"])
result_df = lr_results.join([gp_results, xgb_results, afrr_pr_ts_orig.pd_dataframe()])

In [ ]:
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler
 
 
fig = FigureWidgetResampler(go.Figure())
fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
fig.add_trace(go.Scattergl(name=r'AFRR Up Cap Price', showlegend=True), hf_x=result_df.index, hf_y=result_df['afrr_up_cap_price'])
fig.add_trace(go.Scattergl(name=r'AFRR Up Cap Price Linear Model q = 0.5', showlegend=True), hf_x=result_df.index, hf_y=result_df['gp_afrr_up_cap_price_0.5'])
fig.add_trace(go.Scattergl(name=r'AFRR Up Cap Price Gaussian Process Mean', showlegend=True), hf_x=result_df.index, hf_y=result_df['lr_afrr_up_cap_price_0.5'])
fig.add_trace(go.Scattergl(name=r'AFRR Up Cap Price XGB Mean', showlegend=True), hf_x=result_df.index, hf_y=result_df['xgb_afrr_up_cap_price_0.5'])
fig.update_layout(height=400, template="plotly_dark")
display(fig)

In [ ]:
result_df